<figure>
  <IMG SRC="../img/ENCCS.jpg" WIDTH=50 ALIGN="right">
</figure>

<figure>
  <IMG SRC="../img/VeloxChem.png" WIDTH=100 ALIGN="right">
</figure>

# Møller–Plesset perturbation theory to second order

*Roberto Di Remigio* ([@robertodr](https://github.com/robertodr))

[European National Competence Centre Sweden](https://enccs.se/)

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>